<a href="https://colab.research.google.com/github/shreekrishh/Data-Analytics/blob/main/language%20modelling%20using%20BertForMLM/language_modeling_with_bertMLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Nov  1 13:45:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# #importing drive 
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 40.0 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#importing all necessary libraries
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from torch.nn import functional as F
import torch
from transformers import pipeline
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader
import nltk
import re
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from transformers import AdamW

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = 'cuda'

In [ ]:
# # importing the text file with the encoding
# with open('/gdrive/MyDrive/language modelling/COMBINED (1).txt', 'r', encoding='utf-8') as f:
#   string = f.read()

In [ ]:
# importing the text file with the encoding
with open('/content/drive/MyDrive/language modelling/COMBINED (1).txt', 'r', encoding='utf-8') as f:
  string = f.read()

In [ ]:
# Preprocessing text

def preprocess_text(string):

        text = string.lower()                         #convert the text into lower alphabets
        white = re.compile(r"\n")                     #removing spaces from the string
        text = white.sub(repl="",string = text)
        newline = re.compile(r"\n")                   #replacing new lines with string " " 
        text = newline.sub(repl="",string = text)
        double_newline = re.compile(r"\n\n")
        text = double_newline.sub(repl="",string = text)     
        
        html = re.compile(r'<.*?>')
        text = html.sub(r"",string = text)             #removing html links
        text = re.sub(r'http\S+', '', text)

        sub = sent_tokenize(text)

        for i in tqdm(range(len(sub))):
            pattern = r'[0-9]'                          #removing numbers
            sub[i] = re.sub(pattern, '', sub[i])
            sub[i] = re.sub(r'[^\w\s]', '', sub[i])     #removing punctuation
            sub[i] = sub[i].strip(" ")                  #removing spaces between the quotaions and words
            sub[i] = sub[i] + (".")
            sub[i] = re.sub(' +', ' ', sub[i])
        ignore = []
        for i in range(len(sub)):
            if len(sub[i].split())<3:
                ignore.append(i)
                #sub[i] = "".join(sub[i] + 'IGNORE IGNORE IGNORE')

            if len(sub[i].split())>450:
                ignore.append(i)
                sub[i] = " ".join(sub[i].split()[:450])
        
        
        return sub, ignore

sub, ignore = preprocess_text(string)

100%|██████████| 52749/52749 [00:00<00:00, 57035.37it/s]


In [ ]:
ignore.sort()
count = 0
for i in range(len(ignore)):
  del sub[ignore[i]-count]
  count += 1
len(sub)

46439

In [ ]:
#inherting tokenizer from pretrained bert-base-uncased which contains most of the methods
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = torch.load('/gdrive/MyDrive/language modelling/model.pt')

In [ ]:
ignore_ = []
for i in range(len(sub)):
  if len(tokenizer(sub[i]).input_ids)>500:
    ignore_.append(i)
ignore_.sort(reverse=True)
for i in range(len(ignore_)):
  del sub[ignore_[i]]
len(sub)



Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


46405

In [ ]:
#create a function to drop the last word (here to be gonna predicted) of string 
def drop_last_word(string):
  return string.rsplit(' ',1)[0]
#replacing a mask in place of each target word (that is gonna be predicted)
masked = []
for i in range(len(sub)):
  masked.append(drop_last_word(sub[i]))
  masked[i] += ' [MASK].'
masked[0]

'john f helliwell richard layard jeffrey d sachs janemmanuel de neve lara b aknin and shun wang \x0cthe world happiness report was written by a group of independent experts acting in their personal [MASK].'

In [ ]:
df = pd.DataFrame({'sentence': sub})
df['masked sentence'] = masked
#fucntion for assigning last word of a string as a target word
def target_word(sent):
  return sent.rsplit(' ',1)[1][:-1]
#creating list for target words of all sentences
target = []
for i in range(len(sub)):
  target.append(target_word(sub[i]))
df['Target Words'] = target

In [ ]:
df


,sentence,masked sentence,Target Words
0,john f helliwell richard layard jeffrey d sach...,john f helliwell richard layard jeffrey d sach...,capacities
1,any views expressed in this report do not nece...,any views expressed in this report do not nece...,nations
2,this publication may be reproduced using the f...,this publication may be reproduced using the f...,eds
3,world happiness report new york sustainable de...,world happiness report new york sustainable de...,network
4,full text and supporting documentation can be ...,full text and supporting documentation can be ...,solving
...,...,...,...
46400,this caused me a lot of bewilderment is it pos...,this caused me a lot of bewilderment is it pos...,now
46401,four days ago i got an email from amazon notif...,four days ago i got an email from amazon notif...,wasnt
46402,consequently i called amazon to let them know ...,consequently i called amazon to let them know ...,refunded
46403,i asked for an email documenting the phone con...,i asked for an email documenting the phone con...,day


In [ ]:
df['predictions'] = ''

In [ ]:
df.to_csv("/content/drive/MyDrive/language modelling/sentence_data.csv")

FileNotFoundError: ignored

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/language modelling/sentence_data.csv')

In [ ]:
model.to(device)


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
def inputs(sub, masked_text, x):
  input_text = sub[int(x*len(sub)):int((x+0.8)*len(sub))]
  inputs = tokenizer(input_text, return_tensors='pt', max_length=64, truncation=True, padding='max_length')
  inputs['labels'] = inputs.input_ids.detach().clone()
  inputs_masked = masked[int(x*len(sub)):int((x+0.8)*len(sub))]
  inputs_mask= tokenizer(inputs_masked, return_tensors='pt', max_length=64, truncation=True, padding='max_length')
  inputs.input_ids = inputs_mask.input_ids
  return inputs

In [ ]:
def train(inputs, epochs=3):
  dataset = MeditationsDataset(inputs)
  loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

  for epoch in range(epochs):
      # setup loop with TQDM and dataloader
      loop = tqdm(loader, leave=True)
      for batch in loop:
          
          optim.zero_grad()
        
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)
          # process
          outputs = model(input_ids, attention_mask=attention_mask,
                          labels=labels)
          #extract loss
          loss = outputs.loss
          #calculate loss for every parameter that needs grad update
          loss.backward()
          #update parameters
          optim.step()
          #print relevant info to progress bar
          loop.set_description(f'Epoch {epoch}')
          loop.set_postfix(loss=loss.item())
          model.to(device)
  torch.save(model, '/content/drive/MyDrive/language modelling/model.pt')
 
       

In [ ]:
#train 20% data and save the model and use it for next 20% data, so x=0,0.2,0.4,0.6. after 4 rounds, we will have our model completely trained
inputs_first = inputs(sub, masked, x=0)
train(inputs_first,epochs=3)
# unmasker = pipeline('fill-mask', model= train(), device=0)

  0%|          | 0/2321 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 1:   9%|▉         | 208/2321 [01:45<17:39,  1.99it/s, loss=9.6e-5]

In [ ]:
pred_words_list = []

In [ ]:
def unmask_word(masked_sentence):
  input_ids = tokenizer(masked_sentence,return_tensors='pt', max_length=64, truncation=True, padding='max_length')["input_ids"].to(device)
  #input_ids.to(device)
  logits = model(input_ids)["logits"]
  masked_idx = (input_ids == tokenizer.mask_token_id).to(device)
  #result = logits.argmax(dim=-1).to(device)
  result = torch.topk(logits, 5, dim = -1).indices
  temp_input_ids = input_ids
  flag = False
  for i in range(5):
    temp_input_ids[masked_idx] = result[:, :, i][masked_idx]
    decoded = tokenizer.decode(input_ids[masked_idx])
    if word == decoded: 
      pred_words_list.append(decoded)
      flag = True
      break
    temp_input_ids = input_ids
  if not flag:
      pred_words_list.append('') 
  # input_ids[masked_idx] = result[masked_idx]
  # decoded = tokenizer.decode(input_ids[masked_idx])
  # pred_words_list.append(decoded)
  #result = torch.topk(logits, 5, dim = -1).indices
#   print("Input ids", input_ids.device)
#   print("masked_idx", masked_idx.device)
#   print("result", result.device)
#   for res in result:
#   print(input_ids.size())
#   print(result.size())
#   print("Result", result)
#   print(masked_idx.size())

#   temp_input_ids = input_ids
#   flag = False
#   for i in range(5):
#     temp_input_ids[masked_idx] = result[:, :, i][masked_idx]
#     decoded = tokenizer.decode(input_ids[masked_idx])
#     if word == decoded: 
#         pred_words_list.append(decoded)
#         flag = True
#         break
#     temp_input_ids = input_ids
#   if not flag:
#       pred_words_list.append('') 

In [ ]:
for sentence in tqdm(masked[:int(0.8*len(masked))]):
  unmask_word(sentence)

  0%|          | 0/37124 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
pred_words_list[-10]

''

In [ ]:
target_words = df['Target Words'][:int(0.8*len(df))]
accuracy = 0
for i in range(len(target_words)):
        if pred_words_list[i] == target_words[i]:
            accuracy += 1
accuracy = accuracy / len(pred_words_list)
print("Accuracy of the language model is ",  accuracy)

Accuracy of the language model is  0.2444779657364508


In [ ]:
target_words.shape

(46405,)

In [ ]:
len(pred_words_list)

37124

In [ ]:
print(pred_words_list)

['indefinitely', 'church', 'indefinitely', 'indefinitely', 'indefinitely', 'indefinitely', 'indefinitely', 'indefinitely', 'indefinitely', 'kept', 'kept', 'kept', 'indefinitely', 'kept', 'showed', 'kept', 'called', 'indefinitely', 'indefinitely', 'indefinitely', 'company', 'indefinitely', 'kept', 'built', 'alone', '[PAD]', 'indefinitely', 'indefinitely', 'stopped', 'kept', 'indefinitely', 'indefinitely', 'indefinitely', 'kept', 'indefinitely', 'kept', 'indefinitely', 'indefinitely', 'indefinitely', 'kept', 'indefinitely', 'kept', 'indefinitely', '', 'false', 'followed', 'followed', 'built', 'kept', 'indefinitely', 'indefinitely', 'indefinitely', 'called', 'indefinitely', 'indefinitely', 'indefinitely', 'company', 'indefinitely', 'built', 'indefinitely', 'indefinitely', 'called', 'stopped', 'kept', 'indefinitely', 'built', 'used', 'had', 'indefinitely', 'stopped', 'built', 'based', 'called', 'indefinitely', 'built', 'kept', 'indefinitely', 'kept', 'indefinitely', 'indefinitely', 'kept',

In [ ]:
# def inputs(text, x, y):
#   input_text = sub[int(x*len(sub)):int(y*len(sub))]
#   inputs = tokenizer(input_text, return_tensors='pt', max_length=64, truncation=True, padding='max_length')
#   inputs['labels'] = inputs.input_ids.detach().clone()
#   inputs_masked = masked[int(x*len(sub)):int(y*len(sub))]
#   inputs_mask= tokenizer(inputs_masked, return_tensors='pt', max_length=64, truncation=True, padding='max_length')
#   inputs.input_ids = inputs_mask.input_ids
#   inputs.to(device)
#   return inputs

In [ ]:
def inputs(text, masked_text):
  inputs = tokenizer(text, return_tensors='pt', max_length=64, truncation=True, padding='max_length')
  inputs['labels'] = inputs.input_ids.detach().clone()
  inputs_mask= tokenizer(inputs_masked, return_tensors='pt', max_length=64, truncation=True, padding='max_length')
  inputs.input_ids = inputs_mask.input_ids
  return inputs



In [ ]:
def get_predictions(x,y):
   for sentence in tqdm(sub[int(x*len(sub)):int(y*len(sub))]):

               
                input = tokenizer.encode_plus(sentence, return_tensors = "pt", max_length = 512)            #tokenizing masked sentence
                input.to(device)
                mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)                  
                output = model(**input)                                                                     #output of the model
                logits = output.logits                                                                      #for Prediction of scores of the language modeling for each token
                softmax = F.softmax(logits, dim = -1)                                                     #last activation function for the normalization of the probablility function

                mask_word = softmax[0, mask_index[0], :]
                    
                 
                top = torch.topk(mask_word, 1, dim = 1)[1][0]                                      #predicting the token having highest probability 
                word = tokenizer.decode([top[0]])                                                  #predicting word
                pred_words_list.append(word)
 
  # for i in tqdm(range(len(sub[int(x*len(sub)):int(y*len(sub))]))):
  #   input = inputs(sub[i],masked[i])
  #   input.to(device)
  #   mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)      
  #   output = model(**input)                                                                     #output of the model
  #   logits = output.logits                                                                      #for Prediction of scores of the language modeling for each token
  #   softmax = F.softmax(logits, dim = -1)                                                     #last activation function for the normalization of the probablility function
  #   mask_word = softmax[0, mask_index[0], :]
  #   top_word = torch.argmax(mask_word, dim=1)
  #   pred_words_list.append(tokenizer.decode(top_word))
  #   #top = torch.topk(mask_word, 1, dim = 1)[1][0]  
  #   #word = tokenizer.decode([top[0]])                                                          #predicting word
  #   #pred_words_list.append(word)

        
        
        
        
        
       

# get_predictions()

In [ ]:
get_predictions(0,0.8)

  0%|          | 0/37124 [00:00<?, ?it/s]


IndexError: ignored

In [ ]:
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer, device=0)

In [ ]:
#for i in tqdm(range(len(sub[]))):
pred_words_list.append(unmasker(masked[0])[0]['token_str'])

In [ ]:
pred_words_list = []

In [ ]:
masked_sentences = df['masked sentence']
masked_sentences[0]
len(masked_sentences)

46405

In [ ]:
len(pred_words_list)

46405

In [ ]:
df['predictions'] = pred_words_list
df.to_csv("/content/drive/MyDrive/language modelling/sentence_data.csv")

In [ ]:
df

In [ ]:
# len(pred_words_list)


46409

In [ ]:
# #calculating accuracy
# target_words = df['Target Words']
# accuracy = 0
# for i in range(len(pred_words_list)):
#         if pred_words_list[i] == target_words[i]:
#             accuracy += 1
# accuracy = accuracy / len(target_words)
# # print("Accuracy of the language model is ",  accuracy)

Accuracy of the language model is  0.007665970412799586


In [ ]:
# pred_words_list

In [ ]:
# for i in range(100):
#   df['predictions'][i]=pred_words_list[i]
# 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# display(df[:100])